<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/%5B3rd_type2%5D_3%ED%9A%8C_%EA%B8%B0%EC%B6%9C%EC%9C%A0%ED%98%95_%EC%9E%91%EC%97%85%ED%98%952.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.kaggle.com/competitions/big-data-analytics-certification/code
library(tidyverse) # metapackage of all tidyverse packages
library(dplyr)
library(randomForest)
library(ModelMetrics)
library(nnet)
library(caret)
df_train<-read.csv("../input/big-data-analytics-certification/t2-1-train.csv")
df_test<-read.csv("../input/big-data-analytics-certification/t2-1-test.csv")
sub<-read.csv("../input/big-data-analytics-certification/t2-1-sample_submission.csv")

#################전처리

#id 제거
train<-df_train[,-1]
test<-df_test[,-1]

#결측치 확인후 중간값 대체
train$AnnualIncome<-ifelse(is.na(train$AnnualIncome),median(train$AnnualIncome,na.rm=T),train$AnnualIncome)
test$AnnualIncome<-ifelse(is.na(test$AnnualIncome),median(test$AnnualIncome,na.rm=T),test$AnnualIncome)
#colSums(is.na(df_train))
#colSums(is.na(df_test))
#str(train)
#str(test)


#AnnualIncome min-max scale
p<-preProcess(train,"range")
train$AnnualIncome<-predict(p,train)$AnnualIncome
p1<-preProcess(test,"range")
test$AnnualIncome<-predict(p1,test)$AnnualIncome

#Age구간화 후 factor변경
#Employment.Type,GraduateOrNot,fmailymembers,ChronicDiseases,FrequentFlyer,EverTravelledAborad factor변경
# train<-train%>%mutate(Age=(Age%/%10)*10)%>%mutate_at(vars(-AnnualIncome),as.factor)
# test<-test%>%mutate(Age=(Age%/%10)*10)%>%mutate_at(vars(-AnnualIncome),as.factor)
str(train)
str(test)
# table(test$Employment.Type)
# table(train$Employment.Type)
train<-train%>%mutate_at(vars(-AnnualIncome,-Age),as.factor)
test<-test%>%mutate_at(vars(-AnnualIncome,-Age),as.factor)


# factor중 level 다른것들 일치시켜주기
levels(train$Employment.Type)<-factor(c("Government Sector", "Private Sector/Self Employed","Casual employment"))
#levels(train$Employment.Type)<-c("Casual employment","Government Sector","Private Sector/Self Employed")
#levels(train$Employment.Type)<-levels(test$Employment.Type)
# table(train$Employment.Type)
# table(test$Employment.Type)

# #####모델 선택 
# #train셋 나눔
# set.seed(99)
# idx<-sample(1:nrow(train),0.7*nrow(train))
# x_train<-train[idx,]
# x_test<-train[-idx,]
# str(x_train)
# str(x_test)
# #학습 모델 생성 및 평가

# m_rf<-randomForest(TravelInsurance ~.,x_train,ntree=300)
# p_rf<-predict(m_rf,subset(x_test,select=-TravelInsurance))

# m_nnet<-nnet(TravelInsurance ~.,data=x_train, size = 4)
# p_nnet<-predict(m_nnet,subset(x_test,select=-TravelInsurance),type="class")
# p_nnet<-as.factor(p_nnet)
# str(x_test$TravelInsurance)
# str(p_nnet)
# print("randomForest")
# caret::confusionMatrix(x_test$TravelInsurance,p_rf)$byClass[7]
# caret::confusionMatrix(x_test$TravelInsurance,p_rf)$overall[1]
# cat(paste0("auc:",auc(x_test$TravelInsurance,p_rf)))
# print("nnet")
# caret::confusionMatrix(x_test$TravelInsurance,p_nnet)$byClass[7]
# caret::confusionMatrix(x_test$TravelInsurance,p_nnet)$overall[1]
# cat(paste0("auc:",auc(x_test$TravelInsurance,p_nnet)))

######최종 모델 선택
model_rf<-randomForest(TravelInsurance ~.,train,ntree=300)
pred_rf<-predict(model_rf,test,type="prob")
result <- cbind(df_test[,1],pred_rf[,2])
colnames(result)<-c("id","TravelInsurance")
write.csv(result,"submission.csv",row.names=F)
zz<-read.csv("submission.csv")
head(zz)

